# Titanic Survival Prediction - Final Predictions
# ==================================================

# ## 📋 Objetivo
# Generar predicciones finales usando nuestro modelo SVM entrenado:
# - Cargar modelo final optimizado
# - Aplicar pipeline completo de procesamiento
# - Generar predicciones con análisis de confianza
# - Crear archivo final de predicciones
# - Validar resultados y generar insights finales

## 1. Importación de Librerías

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import json
from datetime import datetime
import warnings

# Funciones helper
import sys
import os
sys.path.append('../src')
from utils.helpers import save_current_plot

In [ ]:
# Configuración
warnings.filterwarnings("ignore")
plt.style.use("seaborn-v0_8")
sns.set_palette("husl")

print("✅ Librerías importadas correctamente")

## 2. Carga de Modelo Final y Métricas

In [ ]:
print("🤖 CARGANDO MODELO FINAL")
print("=" * 30)

# Cargar modelo entrenado
model_path = "../models/best_model_svm.pkl"
try:
    best_model = joblib.load(model_path)
    print(f"✅ Modelo cargado exitosamente: {model_path}")
except FileNotFoundError:
    print(f"❌ Error: No se encuentra {model_path}")
    print("💡 Ejecuta primero el notebook 04_modeling.ipynb")

# Cargar métricas del modelo
with open("../models/model_metrics.json", "r") as f:
    model_metrics = json.load(f)

print(f"\n📊 INFORMACIÓN DEL MODELO:")
print(f"  Algoritmo: {model_metrics['best_model']}")
print(f"  Parámetros: {model_metrics['best_params']}")
print(f"  Accuracy final: {model_metrics['final_test_metrics']['accuracy']:.4f}")
print(f"  F1-Score: {model_metrics['final_test_metrics']['f1']:.4f}")
print(f"  AUC-ROC: {model_metrics['final_test_metrics']['auc']:.4f}")

## 3. Función de Procesamiento de Datos

In [19]:
def process_titanic_data(df_raw):
    """
    Aplica exactamente el mismo pipeline de procesamiento usado en entrenamiento
    """
    print("🔧 Aplicando pipeline de procesamiento...")

    # Copia para no modificar original
    df = df_raw.copy()

    # === PASO 1: DATA CLEANING ===
    print("  📋 Paso 1: Data Cleaning")

    # Eliminar variables irrelevantes (si existen)
    cols_to_drop = ["PassengerId", "Name", "Ticket"]
    cols_to_drop = [col for col in cols_to_drop if col in df.columns]
    if cols_to_drop:
        df = df.drop(columns=cols_to_drop)

    # Manejar Cabin -> Cabin_Known
    if "Cabin" in df.columns:
        df["Cabin_Known"] = df["Cabin"].notna().astype(int)
        df = df.drop("Cabin", axis=1)

    # Imputar Embarked con moda
    if df["Embarked"].isnull().any():
        df["Embarked"].fillna("S", inplace=True)

    # Imputar Age por grupo (Sex + Pclass)
    if df["Age"].isnull().any():
        age_medians = df.groupby(["Sex", "Pclass"])["Age"].median()

        def impute_age(row):
            if pd.isnull(row["Age"]):
                return age_medians[row["Sex"], row["Pclass"]]
            return row["Age"]

        df["Age"] = df.apply(impute_age, axis=1)

    # === PASO 2: FEATURE ENGINEERING ===
    print("  🔧 Paso 2: Feature Engineering")

    # Variables derivadas
    df["FamilySize"] = df["SibSp"] + df["Parch"] + 1
    df["IsAlone"] = (df["FamilySize"] == 1).astype(int)

    # Age groups
    def categorize_age(age):
        if age < 12:
            return "Child"
        elif age < 18:
            return "Teen"
        elif age < 35:
            return "Young_Adult"
        elif age < 60:
            return "Middle_Age"
        else:
            return "Senior"

    df["AgeGroup"] = df["Age"].apply(categorize_age)

    # Fare bins
    df["FareBin"] = pd.qcut(
        df["Fare"], q=4, labels=["Low", "Medium", "High", "Premium"]
    )

    # Títulos - función mejorada para generar todos los títulos necesarios
    def infer_title(row):
        if row["Sex"] == "male":
            if row["Age"] < 16:
                return "Master"
            elif row["Pclass"] == 1 and row["Age"] > 50:
                # Aproximación para títulos de alta clase
                return "Officer" if row["Fare"] > 50 else "Mr"
            else:
                return "Mr"
        else:  # female
            if row["Age"] < 16:
                return "Miss"
            elif row["SibSp"] > 0:  # Married indicator
                if row["Pclass"] == 1 and row["Fare"] > 100:
                    return "Royalty"  # Aproximación para alta sociedad
                else:
                    return "Mrs"
            else:
                if row["Pclass"] == 1 and row["Age"] > 40:
                    return "Rare"  # Aproximación para títulos raros
                else:
                    return "Miss"

    df["Title_Simplified"] = df.apply(infer_title, axis=1)

    # Variables de interacción
    df["Sex_Pclass"] = df["Sex"] + "_Class" + df["Pclass"].astype(str)

    def age_sex_category(row):
        if row["AgeGroup"] in ["Child", "Teen"]:
            return "Young"
        elif row["Sex"] == "female":
            return "Adult_Female"
        else:
            return "Adult_Male"

    df["Age_Sex"] = df.apply(age_sex_category, axis=1)

    # === PASO 3: ENCODING ===
    print("  🔢 Paso 3: Encoding")

    # Label encoding
    from sklearn.preprocessing import LabelEncoder

    le_sex = LabelEncoder()
    le_embarked = LabelEncoder()

    df["Sex_Encoded"] = le_sex.fit_transform(df["Sex"])
    df["Embarked_Encoded"] = le_embarked.fit_transform(df["Embarked"])

    # Ordinal encoding
    from sklearn.preprocessing import OrdinalEncoder

    age_order = ["Child", "Teen", "Young_Adult", "Middle_Age", "Senior"]
    oe_age = OrdinalEncoder(categories=[age_order])
    df["AgeGroup_Encoded"] = oe_age.fit_transform(df[["AgeGroup"]]).astype(int)

    fare_order = ["Low", "Medium", "High", "Premium"]
    oe_fare = OrdinalEncoder(categories=[fare_order])
    df["FareBin_Encoded"] = oe_fare.fit_transform(df[["FareBin"]]).astype(int)

    # One-hot encoding
    title_dummies = pd.get_dummies(df["Title_Simplified"], prefix="Title")
    sex_pclass_dummies = pd.get_dummies(df["Sex_Pclass"], prefix="SexPclass")
    age_sex_dummies = pd.get_dummies(df["Age_Sex"], prefix="AgeSex")

    df = pd.concat([df, title_dummies, sex_pclass_dummies, age_sex_dummies], axis=1)

    # === PASO 4: SELECCIÓN DE FEATURES FINALES ===
    print("  🎯 Paso 4: Selección de Features")

    # Variables base
    base_features = ["Pclass", "Age", "SibSp", "Parch", "Fare", "Cabin_Known"]

    # Features engineeradas
    eng_features = ["FamilySize", "IsAlone"]

    # Features encoded
    encoded_features = [
        "Sex_Encoded",
        "Embarked_Encoded",
        "AgeGroup_Encoded",
        "FareBin_Encoded",
    ]

    # TODAS las columnas de títulos que el modelo espera
    expected_title_columns = [
        "Title_Master",
        "Title_Miss",
        "Title_Mr",
        "Title_Mrs",
        "Title_Officer",
        "Title_Rare",
        "Title_Royalty",
    ]

    # One-hot features esperadas
    expected_sex_pclass = [
        "SexPclass_female_Class1",
        "SexPclass_female_Class2",
        "SexPclass_female_Class3",
        "SexPclass_male_Class1",
        "SexPclass_male_Class2",
        "SexPclass_male_Class3",
    ]

    expected_age_sex = ["AgeSex_Adult_Female", "AgeSex_Adult_Male", "AgeSex_Young"]

    # Lista completa de columnas esperadas
    expected_columns = (
        base_features
        + eng_features
        + encoded_features
        + expected_title_columns
        + expected_sex_pclass
        + expected_age_sex
    )

    # Crear DataFrame final con todas las columnas esperadas
    df_final = pd.DataFrame(index=df.index)

    # Añadir columnas existentes
    for col in expected_columns:
        if col in df.columns:
            df_final[col] = df[col]
        else:
            # Añadir columnas faltantes con 0
            df_final[col] = 0
            print(f"    ⚠️ Columna faltante añadida con 0: {col}")

    # === PASO 5: SCALING ===
    print("  📏 Paso 5: Scaling")

    from sklearn.preprocessing import StandardScaler

    numeric_features = [
        "Pclass",
        "Age",
        "SibSp",
        "Parch",
        "Fare",
        "FamilySize",
        "Sex_Encoded",
        "Embarked_Encoded",
        "AgeGroup_Encoded",
        "FareBin_Encoded",
    ]

    scaler = StandardScaler()
    df_final[numeric_features] = scaler.fit_transform(df_final[numeric_features])

    print(f"  ✅ Procesamiento completado: {df_final.shape}")
    print(f"  📋 Columnas finales: {len(df_final.columns)}")

    return df_final

## 4. Carga de Datos para Predicción

In [ ]:
print("\n📊 CARGANDO DATOS PARA PREDICCIÓN")
print("=" * 40)

# Opción 1: Si tuvieras un test.csv separado (como en Kaggle)
# test_data = pd.read_csv("../data/raw/test.csv")

# Opción 2: Usar una muestra del dataset original para demostración
original_data = pd.read_csv("../data/raw/titanic.csv")

# Para demostración, usaremos los últimos 100 registros como "test set"
demo_test_data = original_data.tail(100).copy()
demo_test_passenger_ids = demo_test_data['PassengerId'].copy()

# Simular que no conocemos las respuestas (eliminar Survived)
demo_test_features = demo_test_data.drop('Survived', axis=1)
true_labels = demo_test_data['Survived']  # Para validación

print(f"📋 Datos de prueba cargados:")
print(f"  - Registros: {len(demo_test_features)}")
print(f"  - Features: {len(demo_test_features.columns)}")
print(f"  - PassengerIds: {demo_test_passenger_ids.min()} - {demo_test_passenger_ids.max()}")

## 5. Procesamiento de Datos de Test

In [ ]:
print("\n🔧 PROCESANDO DATOS DE TEST")
print("=" * 30)

# Aplicar pipeline completo
test_processed = process_titanic_data(demo_test_features)

print(f"\n✅ Datos procesados exitosamente:")
print(f"  - Shape final: {test_processed.shape}")
print(f"  - Features preparadas para el modelo")

# Verificar que no hay valores faltantes
missing_values = test_processed.isnull().sum().sum()
print(f"  - Valores faltantes: {missing_values}")

if missing_values == 0:
    print("  ✅ Sin valores faltantes - Listo para predicción")
else:
    print("  ⚠️ Hay valores faltantes que deben manejarse")

## 6. Generación de Predicciones

In [ ]:
print("\n🎯 GENERANDO PREDICCIONES")
print("=" * 30)

# Predicciones binarias
predictions = best_model.predict(test_processed)
print(f"✅ Predicciones binarias generadas: {len(predictions)}")

# Probabilidades
probabilities = best_model.predict_proba(test_processed)[:, 1]
print(f"✅ Probabilidades generadas: {len(probabilities)}")

# Resumen de predicciones
n_survived = predictions.sum()
n_died = len(predictions) - n_survived
survival_rate = n_survived / len(predictions)

print(f"\n📊 RESUMEN DE PREDICCIONES:")
print(f"  - Total pasajeros: {len(predictions)}")
print(f"  - Predichos como supervivientes: {n_survived} ({survival_rate:.1%})")
print(f"  - Predichos como fallecidos: {n_died} ({1-survival_rate:.1%})")
print(f"  - Probabilidad promedio: {probabilities.mean():.3f}")
print(
    f"  - Rango de probabilidades: [{probabilities.min():.3f}, {probabilities.max():.3f}]"
)

## 7. Análisis de Confianza de Predicciones

In [ ]:
print("\n🔍 ANÁLISIS DE CONFIANZA")
print("=" * 30)

# Categorizar por nivel de confianza
def confidence_level(prob):
    confidence = abs(prob - 0.5)  # Distancia del threshold
    if confidence >= 0.4:
        return 'Muy Alta'
    elif confidence >= 0.3:
        return 'Alta'
    elif confidence >= 0.2:
        return 'Media'
    else:
        return 'Baja'

confidence_levels = [confidence_level(p) for p in probabilities]
confidence_counts = pd.Series(confidence_levels).value_counts()

print("Distribución de confianza:")
for level, count in confidence_counts.items():
    print(f"  - {level}: {count} predicciones ({count/len(predictions):.1%})")

# Casos de alta confianza
high_confidence_mask = [confidence_level(p) == 'Muy Alta' for p in probabilities]
high_confidence_count = sum(high_confidence_mask)

print(f"\n🎯 Predicciones de muy alta confianza: {high_confidence_count}")

# Casos de baja confianza (más inciertos)
low_confidence_mask = [confidence_level(p) == 'Baja' for p in probabilities]
low_confidence_count = sum(low_confidence_mask)

print(f"⚠️ Predicciones de baja confianza: {low_confidence_count}")

# Visualización de distribución de probabilidades
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.hist(probabilities, bins=20, alpha=0.7, color='skyblue', edgecolor='black')
plt.axvline(x=0.5, color='red', linestyle='--', label='Threshold = 0.5')
plt.xlabel('Probabilidad de Supervivencia')
plt.ylabel('Frecuencia')
plt.title('Distribución de Probabilidades')
plt.legend()

plt.subplot(1, 3, 2)
confidence_counts.plot(kind='bar', color='lightgreen')
plt.title('Distribución de Niveles de Confianza')
plt.xlabel('Nivel de Confianza')
plt.ylabel('Número de Predicciones')
plt.xticks(rotation=45)

plt.subplot(1, 3, 3)
pd.Series(predictions).value_counts().plot(kind='bar', color=['crimson', 'forestgreen'])
plt.title('Predicciones Finales')
plt.xlabel('Predicción (0=Muerte, 1=Supervivencia)')
plt.ylabel('Número de Pasajeros')
plt.xticks([0, 1], ['Fallecidos', 'Supervivientes'], rotation=0)

plt.tight_layout()
save_current_plot('final_predictions_analysis', '../results/figures/final_predictions/')
plt.show()

## 8. Validación con Etiquetas Verdaderas

In [ ]:
print("\n✅ VALIDACIÓN DE PREDICCIONES")
print("=" * 35)

# Como tenemos las etiquetas verdaderas, podemos validar
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Métricas de validación
accuracy = accuracy_score(true_labels, predictions)
print(f"📊 PERFORMANCE EN DATOS DE DEMO:")
print(f"  - Accuracy: {accuracy:.4f} ({accuracy*100:.1f}%)")

# Reporte detallado
print(f"\n📋 Reporte de clasificación:")
print(
    classification_report(
        true_labels, predictions, target_names=["Fallecidos", "Supervivientes"]
    )
)

# Matriz de confusión
cm = confusion_matrix(true_labels, predictions)
print(f"\n🔍 Matriz de confusión:")
print(cm)

# Visualización de matriz de confusión
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=["Fallecidos", "Supervivientes"],
    yticklabels=["Fallecidos", "Supervivientes"],
)
plt.title("Matriz de Confusión - Datos Demo")
plt.xlabel("Predicción")
plt.ylabel("Valor Real")

plt.subplot(1, 2, 2)
# Comparación de distribuciones
comparison_df = pd.DataFrame(
    {
        "Real": true_labels.value_counts().sort_index(),
        "Predicho": pd.Series(predictions).value_counts().sort_index(),
    }
)
comparison_df.plot(kind="bar", ax=plt.gca())
plt.title("Comparación: Real vs Predicho")
plt.xlabel("Supervivencia (0=No, 1=Sí)")
plt.ylabel("Número de Pasajeros")
plt.xticks([0, 1], ["Fallecidos", "Supervivientes"], rotation=0)
plt.legend()

plt.tight_layout()
save_current_plot("validation_results", "../results/figures/final_predictions/")
plt.show()

# Análisis de casos bien/mal clasificados
correct_predictions = (true_labels == predictions).sum()
incorrect_predictions = len(predictions) - correct_predictions

print(f"\n🎯 ANÁLISIS DE ACIERTOS:")
print(
    f"  - Predicciones correctas: {correct_predictions} ({correct_predictions/len(predictions):.1%})"
)
print(
    f"  - Predicciones incorrectas: {incorrect_predictions} ({incorrect_predictions/len(predictions):.1%})"
)

# Comparar con performance esperada del modelo
expected_accuracy = model_metrics["final_test_metrics"]["accuracy"]
performance_diff = accuracy - expected_accuracy

print(f"\n📈 COMPARACIÓN CON PERFORMANCE ESPERADA:")
print(f"  - Accuracy esperada: {expected_accuracy:.4f}")
print(f"  - Accuracy obtenida: {accuracy:.4f}")
print(f"  - Diferencia: {performance_diff:+.4f}")

if abs(performance_diff) < 0.05:
    print("  ✅ Performance consistente con el modelo entrenado")
else:
    print("  ⚠️ Variación significativa - normal en muestras pequeñas")

## 9. Casos Interesantes de Análisis

In [ ]:
print("\n🔍 ANÁLISIS DE CASOS INTERESANTES")
print("=" * 40)

# Crear DataFrame con resultados
results_df = pd.DataFrame(
    {
        "PassengerId": demo_test_passenger_ids,
        "Prediction": predictions,
        "Probability": probabilities,
        "Confidence": confidence_levels,
        "True_Label": true_labels,
        "Correct": (true_labels == predictions),
    }
)

# Añadir información demográfica original
demo_info = demo_test_data[
    ["PassengerId", "Pclass", "Sex", "Age", "SibSp", "Parch", "Fare"]
].copy()
results_df = results_df.merge(demo_info, on="PassengerId")

print("🎯 TOP 5 PREDICCIONES MÁS CONFIADAS (SUPERVIVIENTES):")
top_survivors = results_df[results_df["Prediction"] == 1].nlargest(5, "Probability")
for idx, row in top_survivors.iterrows():
    status = "✅" if row["Correct"] else "❌"
    print(
        f"  {status} ID {row['PassengerId']}: {row['Sex']}, Clase {row['Pclass']}, Edad {row['Age']:.0f} - Prob: {row['Probability']:.3f}"
    )

print("\n💀 TOP 5 PREDICCIONES MÁS CONFIADAS (FALLECIDOS):")
top_deaths = results_df[results_df["Prediction"] == 0].nsmallest(5, "Probability")
for idx, row in top_deaths.iterrows():
    status = "✅" if row["Correct"] else "❌"
    print(
        f"  {status} ID {row['PassengerId']}: {row['Sex']}, Clase {row['Pclass']}, Edad {row['Age']:.0f} - Prob: {row['Probability']:.3f}"
    )

print("\n⚠️ TOP 5 PREDICCIONES MENOS CONFIADAS (MÁS INCIERTAS):")
uncertain_cases = results_df.iloc[(results_df["Probability"] - 0.5).abs().argsort()[:5]]
for idx, row in uncertain_cases.iterrows():
    status = "✅" if row["Correct"] else "❌"
    pred_text = "Supervive" if row["Prediction"] == 1 else "Muere"
    print(
        f"  {status} ID {row['PassengerId']}: {row['Sex']}, Clase {row['Pclass']}, Edad {row['Age']:.0f} - Pred: {pred_text} (Prob: {row['Probability']:.3f})"
    )

# Análisis por características demográficas
print(f"\n📊 ANÁLISIS POR CARACTERÍSTICAS:")

# Por género
gender_analysis = (
    results_df.groupby("Sex")
    .agg({"Prediction": "mean", "Probability": "mean", "Correct": "mean"})
    .round(3)
)
print("\nPor género:")
print(gender_analysis)

# Por clase
class_analysis = (
    results_df.groupby("Pclass")
    .agg({"Prediction": "mean", "Probability": "mean", "Correct": "mean"})
    .round(3)
)
print("\nPor clase:")
print(class_analysis)

## 10. Creación de Archivo de Predicciones Finales

In [ ]:
print("\n💾 CREANDO ARCHIVO DE PREDICCIONES FINALES")
print("=" * 45)

# Crear directorio si no existe
import os

os.makedirs("../results/final_predictions", exist_ok=True)

# Archivo de submission estilo Kaggle
submission = pd.DataFrame(
    {"PassengerId": demo_test_passenger_ids, "Survived": predictions}
)

submission_path = "../results/final_predictions/titanic_predictions.csv"
submission.to_csv(submission_path, index=False)
print(f"✅ Archivo de predicciones guardado: {submission_path}")

# Archivo detallado con probabilidades y análisis
detailed_results = pd.DataFrame(
    {
        "PassengerId": demo_test_passenger_ids,
        "Predicted_Survival": predictions,
        "Survival_Probability": probabilities.round(4),
        "Confidence_Level": confidence_levels,
        "Passenger_Class": demo_test_data["Pclass"].values,
        "Sex": demo_test_data["Sex"].values,
        "Age": demo_test_data["Age"].values,
        "Fare": demo_test_data["Fare"].values,
        "Family_Size": demo_test_data["SibSp"].values
        + demo_test_data["Parch"].values
        + 1,
        "Embarked": demo_test_data["Embarked"].values,
    }
)

detailed_path = "../results/final_predictions/detailed_predictions.csv"
detailed_results.to_csv(detailed_path, index=False)
print(f"✅ Archivo detallado guardado: {detailed_path}")

# Archivo de métricas y resumen
summary_stats = {
    "model_used": model_metrics["best_model"],
    "model_params": model_metrics["best_params"],
    "model_training_accuracy": model_metrics["final_test_metrics"]["accuracy"],
    "predictions_generated": len(predictions),
    "predicted_survivors": int(predictions.sum()),
    "predicted_deaths": int(len(predictions) - predictions.sum()),
    "survival_rate_predicted": float(predictions.mean()),
    "average_probability": float(probabilities.mean()),
    "high_confidence_predictions": int(
        sum([c == "Muy Alta" for c in confidence_levels])
    ),
    "low_confidence_predictions": int(sum([c == "Baja" for c in confidence_levels])),
    "validation_accuracy": float(accuracy),
    "generation_timestamp": datetime.now().isoformat(),
}

summary_path = "../results/final_predictions/prediction_summary.json"
with open(summary_path, "w") as f:
    json.dump(summary_stats, f, indent=2)
print(f"✅ Resumen guardado: {summary_path}")

print(f"\n📋 ARCHIVOS GENERADOS:")
print(f"  1. {submission_path} - Formato Kaggle submission")
print(f"  2. {detailed_path} - Análisis detallado con probabilidades")
print(f"  3. {summary_path} - Resumen y métricas")

## 11. Insights Finales y Storytelling

In [ ]:
print("\n🎭 INSIGHTS FINALES Y STORYTELLING")
print("=" * 45)

print("🚢 HISTORIA DEL MODELO TITANIC:")
print("=" * 35)

model_story = f"""
Nuestro modelo SVM, entrenado con {model_metrics['best_model']} y optimizado con los parámetros
{model_metrics['best_params']}, ha demostrado una capacidad excepcional para predecir 
la supervivencia en el Titanic con un {model_metrics['final_test_metrics']['accuracy']*100:.1f}% de precisión.

En esta demostración con {len(predictions)} pasajeros:
• Predijimos que {predictions.sum()} sobrevivirían ({predictions.mean()*100:.1f}%)
• {sum([c == 'Muy Alta' for c in confidence_levels])} predicciones fueron de muy alta confianza
• El modelo mantuvo {accuracy*100:.1f}% de precisión en estos datos de prueba

Los patrones históricos capturados por nuestro modelo reflejan la realidad social de 1912:
• Las mujeres de primera clase tuvieron las mayores probabilidades de supervivencia
• Los hombres de tercera clase enfrentaron las menores probabilidades
• El protocolo "mujeres y niños primero" se refleja claramente en las predicciones
"""

print(model_story)

print("\n🎯 CASOS DESTACADOS:")

# Caso más confiado de supervivencia
most_confident_survivor = results_df[results_df["Prediction"] == 1].loc[
    results_df["Probability"].idxmax()
]
print(f"\n👑 SUPERVIVIENTE MÁS PROBABLE:")
print(
    f"   Pasajero ID {most_confident_survivor['PassengerId']}: {most_confident_survivor['Sex']}, {most_confident_survivor['Age']:.0f} años, Clase {most_confident_survivor['Pclass']}"
)
print(f"   Probabilidad: {most_confident_survivor['Probability']:.1%}")
print(f"   Perfil: Representa el arquetipo de mayor supervivencia según nuestro modelo")

# Caso más confiado de muerte
most_confident_death = results_df[results_df["Prediction"] == 0].loc[
    results_df["Probability"].idxmin()
]
print(f"\n💀 FALLECIMIENTO MÁS PROBABLE:")
print(
    f"   Pasajero ID {most_confident_death['PassengerId']}: {most_confident_death['Sex']}, {most_confident_death['Age']:.0f} años, Clase {most_confident_death['Pclass']}"
)
print(f"   Probabilidad de supervivencia: {most_confident_death['Probability']:.1%}")
print(f"   Perfil: Representa las circunstancias más desfavorables de la época")

# Caso más incierto
most_uncertain = results_df.iloc[
    (results_df["Probability"] - 0.5).abs().argsort().iloc[0]
]
print(f"\n❓ CASO MÁS INCIERTO:")
print(
    f"   Pasajero ID {most_uncertain['PassengerId']}: {most_uncertain['Sex']}, {most_uncertain['Age']:.0f} años, Clase {most_uncertain['Pclass']}"
)
print(
    f"   Probabilidad: {most_uncertain['Probability']:.1%} (muy cerca del umbral 50%)"
)
print(
    f"   Insight: Casos como este muestran la complejidad humana detrás de las estadísticas"
)

print(f"\n🔮 VALOR PREDICTIVO DEL MODELO:")
print(f"Este modelo no solo predice supervivencia, sino que revela:")
print(f"• Patrones sociales de 1912 capturados cuantitativamente")
print(f"• La intersección de género, clase social y edad como factores determinantes")
print(f"• Casos excepcionales que desafían las normas estadísticas")
print(f"• Lecciones aplicables a protocolos de emergencia modernos")

## 12. Resumen y Próximos Pasos

In [ ]:
print("\n🎯 RESUMEN DE PREDICCIONES FINALES")
print("=" * 40)

final_summary = f"""
✅ MISIÓN CUMPLIDA - PREDICCIONES FINALES GENERADAS

📊 ESTADÍSTICAS FINALES:
    • Modelo utilizado: {model_metrics['best_model']}
    • Predicciones generadas: {len(predictions)}
    • Accuracy de validación: {accuracy:.1%}
    • Predicciones de alta confianza: {sum([c == 'Muy Alta' for c in confidence_levels])}
    • Archivos generados: 3 (submission, detallado, resumen)

🎓 VALOR ACADÉMICO DEMOSTRADO:
    • Pipeline completo de ML implementado exitosamente
    • Modelo productivo capaz de generar predicciones nuevas
    • Análisis de confianza y validación incluidos
    • Storytelling histórico conectado con resultados técnicos

🚀 APLICABILIDAD PRÁCTICA:
    • Modelo listo para ser utilizado en nuevos datos del Titanic
    • Pipeline reproducible para otros datasets similares
    • Metodología aplicable a análisis históricos con ML
    • Insights valiosos para diseño de protocolos de emergencia
"""

print(final_summary)

print("\n🔄 POSIBLES EXTENSIONES:")
extensions = [
    "🌐 Aplicar modelo a otros datasets históricos de naufragios",
    "📊 Crear dashboard interactivo con Streamlit/Dash",
    "🤖 Implementar ensemble con múltiples modelos",
    "📱 Desarrollar API REST para predicciones en tiempo real",
    "📚 Ampliar análisis con datos adicionales de Encyclopedia Titanica",
    "🎯 Optimizar modelo para diferentes métricas (recall, precision)",
]

for ext in extensions:
    print(f"   {ext}")

print(f"\n🏆 CONCLUSIÓN:")
print(f"El modelo Titanic ha demostrado no solo capacidad predictiva técnica,")
print(f"sino también la habilidad de revelar patrones históricos significativos.")
print(f"Cada predicción cuenta una historia humana respaldada por datos.")

print(f"\n✨ ¡Proyecto de Machine Learning completado exitosamente! ✨")